### Messages in Autogen v0.4
- We can imaging messages as the way agent communicate - text our Friend.
- When we communicate with the agents -----> sending a message when it responds ---> it too sends a message
- TextMessage ImageMessage ToolMessage

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage

from PIL import Image
from io import BytesIO
import requests

In [3]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
    model_info={
        "family": "gemini-1.5-flash",
        "vision": True,
        "function_calling": True,
        "json_output": False,
        "structured_output": True
    }
)

In [4]:
model_client

#### Simplest Type of Message - TextMessage

In [5]:
agent = AssistantAgent(
    name="Text_Agent",
    model_client=model_client,
    system_message='You are a helpful assistant. answer question accurately'
)

In [6]:
agent

In [8]:
async def test_text_messages():
    task = TextMessage(content="Expalin me in Details Fast Fourier Transform?", source="user")
    response = await agent.run(task=task)
    print(response.messages[-1].content)

await test_text_messages()

The Fast Fourier Transform (FFT) is an algorithm that computes the Discrete Fourier Transform (DFT) significantly faster than a direct evaluation of the DFT formula.  The DFT is a mathematical procedure that decomposes a sequence of values (often representing a signal sampled in time or space) into its constituent frequencies.  A direct DFT computation has a time complexity of O(N²), where N is the number of data points.  This becomes prohibitively slow for large datasets.  The FFT reduces this complexity to O(N log N), making it feasible to analyze much larger signals.

Let's break down the key aspects:

**1. The Discrete Fourier Transform (DFT):**

The DFT takes a sequence of *N* complex numbers,  `x[0], x[1], ..., x[N-1]`, and transforms it into another sequence of *N* complex numbers, `X[0], X[1], ..., X[N-1]`, representing the frequency components.  The formula is:

`X[k] = Σ_(n=0)^(N-1) x[n] * e^(-j2πkn/N)`

where:

* `k` is the frequency index (0 ≤ k ≤ N-1).  `k=0` represents th

#### More Complicated Message Than TextMessage - MultiModalMessage

In [9]:
async def test_multi_model():
    response = requests.get("https://picsum.photos/id/15/200/300")  # 23 for the image of folks
    pil_image = Image.open(BytesIO(response.content))
    ag_image = AGImage(pil_image)
    
    multi_model_msg = MultiModalMessage(
        content=["What is in the image?", ag_image],
        source="user"
    )
    
    result = await agent.run(task=multi_model_msg)
    
    print(result.messages[-1].content)

await test_multi_model()

That's an image of a waterfall cascading down a steep, rocky cliff face.  The scene is lush and green, indicating a moist, possibly rainforest-like environment.  The foreground shows dark, slate-like rocks and pebbles at the base of the waterfall.  The water appears to pool slightly at the bottom before continuing its flow.  The overall impression is one of a secluded, natural, and possibly somewhat remote location.

